In [1]:
%matplotlib inline
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import OneHotEncoder, RobustScaler, MinMaxScaler
from sklearn import preprocessing

import matplotlib.image as mpimg
from skimage.io import imread, imshow
from skimage import data, color, io, filters, morphology,transform, exposure, feature, util
from scipy import ndimage
#import Tensorflow namespaces

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image

from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input, decode_predictions

import pickle

#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
from os import walk, path
from random import shuffle
from skimage.io import imread, imshow
from skimage import data, color, io, filters, morphology,transform, exposure, feature, util
import matplotlib.pyplot as plt
import numpy as np
import glob, random

In [3]:
def load_images(path):
    
    images = []
    for image_file in os.listdir(path):

        image = io.imread(os.path.join(path, image_file))
        if image is not None:
            images.append(image)
            

    return images

cars = load_images('./vehicles')
bikes = load_images('./bike')
non_vehicles = load_images("./non-vehicles")

In [8]:
img_rows, img_cols = 64, 64
car_img_res = []
for i in range(0,len(cars)):
        car_img_res.append(transform.resize(cars[i],(img_rows,img_cols,3), mode='constant'))


In [9]:

bikes_img_res = []
for i in range(0,len(bikes)):
        bikes_img_res.append(transform.resize(bikes[i],(img_rows,img_cols,3), mode='constant'))

In [10]:
print("Length of the datasets:")
print(f"{len(car_img_res)}, {len(bikes_img_res)}")

Length of the datasets:
420, 365


In [38]:
Xs = car_img_res + bikes_img_res + non_vehicles[:len(car_img_res)*2]
Xs = np.array(Xs)

Xs = Xs.astype('float32')
Xs /= 255

In [39]:
print(Xs.shape)

(1625, 64, 64, 3)


In [40]:
# Generate labels 2 for bikes, 1 for vehicles and 0 for non-vehicles
class_labels = np.array([2] * len(car_img_res)+ [1] * len(bikes_img_res) + [0] * (len(car_img_res)*2))

In [41]:
# Devide the Xs into training and test sets
X_train, X_test, y_train, y_test = train_test_split(Xs, class_labels, test_size=0.2, random_state=20)

y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1300, 64, 64, 3)
(325, 64, 64, 3)
(1300, 3)
(325,)


In [42]:
adam = tf.keras.optimizers.Adam(learning_rate = 0.0001)

# Neural network parameters
#-----------------------------------------------
#-----------------------------------------------
batch_size = 64 #
epochs = 50 # 

kernel_size=(3,3)
#-----------------------------------------------
#-----------------------------------------------
num_classes = 3
input_shape = ( X_train.shape[1], X_train.shape[2], X_train.shape[3])


# Model
model = Sequential()
model.add(Conv2D(16, kernel_size=kernel_size, input_shape=input_shape)) 
model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(16, kernel_size=kernel_size)) 

model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))


model.add(Flatten()) 
model.add(Dropout(0.4)) 
model.add(Dense(20, activation='relu')) 

model.add(Dropout(0.4))

model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [43]:
# Trainen van het CNN
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, min_delta=0.001, patience=3, restore_best_weights=True)

history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2, verbose=1, callbacks=[es])


Epoch 1/50
17/17 [==============================] - 1s 31ms/step - loss: 1.0827 - accuracy: 0.4779 - val_loss: 1.0409 - val_accuracy: 0.5154
Epoch 2/50
17/17 [==============================] - 0s 13ms/step - loss: 0.7903 - accuracy: 0.5981 - val_loss: 1.0081 - val_accuracy: 0.5154
Epoch 3/50
17/17 [==============================] - 0s 12ms/step - loss: 0.6919 - accuracy: 0.6423 - val_loss: 0.9632 - val_accuracy: 0.5154
Epoch 4/50
17/17 [==============================] - 0s 12ms/step - loss: 0.6090 - accuracy: 0.6962 - val_loss: 0.9067 - val_accuracy: 0.5154
Epoch 5/50
17/17 [==============================] - 0s 12ms/step - loss: 0.5705 - accuracy: 0.6990 - val_loss: 0.8517 - val_accuracy: 0.5154
Epoch 6/50
17/17 [==============================] - 0s 13ms/step - loss: 0.5520 - accuracy: 0.7221 - val_loss: 0.7941 - val_accuracy: 0.5154
Epoch 7/50
17/17 [==============================] - 0s 12ms/step - loss: 0.5179 - accuracy: 0.7288 - val_loss: 0.7359 - val_accuracy: 0.5154
Epoch 8/50
17

In [44]:
y_predicted = np.argmax(model.predict(X_test),axis=-1)

print(classification_report(y_test, y_predicted))

cf = confusion_matrix(y_test, y_predicted)

print(cf)
print(accuracy_score(y_test, y_predicted) * 100) 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       168
           1       0.00      0.00      0.00        74
           2       0.53      1.00      0.69        83

    accuracy                           0.77       325
   macro avg       0.51      0.67      0.56       325
weighted avg       0.65      0.77      0.69       325

[[168   0   0]
 [  0   0  74]
 [  0   0  83]]
77.23076923076924


d:\Howest 21-22\ResearchProjectGH\NeuralNetworks\rpvenv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Howest 21-22\ResearchProjectGH\NeuralNetworks\rpvenv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Howest 21-22\ResearchProjectGH\NeuralNetworks\rpvenv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera